In [1]:
import pennylane as qml
from pennylane import numpy as np
from pennylane.optimize import AdamOptimizer, GradientDescentOptimizer

import matplotlib.pyplot as plt
from matplotlib import colors

In [2]:
def embedding_layer(X):
    for i in range(X.shape[0]):
        qml.RZ(-1*X[i,2],wires=i)
        qml.RY(X[i,0],wires=i)
        qml.RX(X[i,1],wires=i)
        qml.RZ(X[i,2],wires=i)
        
def rbs(theta, wires):
    a = np.cos(theta)
    b = np.sin(theta)
    U = np.array([[1, 0, 0, 0], [0, a, b, 0], [0, -b, a, 0], [0, 0, 0, 1]], requires_grad=True)
    qml.QubitUnitary(U, wires=wires)
        
def XY_layer(N,params):
    for i in range(N-1):
#         qml.IsingXY(params[i],wires = [i,i+1])
#     qml.IsingXY(params[N-1],wires = [N-1,0])
        rbs(params[i],wires = [i,i+1])
    rbs(params[N-1],wires = [N-1,0])
    
def Y_pooling(N):
    n2 = N//2
    for i in range(n2):
        qml.CY(wires=[i+n2, i])

In [15]:
dev = qml.device("default.qubit", wires=10)

def process_data2(Data):
    Data[:,:,0]*=(1/250)
    Index = Data[:,:,2]-2*np.pi>0
    Data[Index,2]-= 2*np.pi
    Index2 = Data[:,:,2]<0
    Data[Index2,2]+= 2*np.pi
    Data[:,:,2]-=np.pi
    return np.array(Data[:,:,:3])


Path =  '../data/'
Data = np.load(Path+'Sorted_10_data.npy')
Label = np.load(Path+'Sorted_10_label.npy')
Data = process_data2(Data)
Label = np.array(Label).astype(int)

Data = np.array(Data, requires_grad=False)
Label = np.array(Label, requires_grad=False)

idx = np.arange(Data.shape[0])
np.random.shuffle(idx)
Data,Label = Data[idx], Label[idx]

Train_data, Train_label = Data[:700,:10,:], Label[:700]
Test_data, Test_label = Data[700:800,:10,:], Label[700:800]

Train_data, Train_label = np.array(Train_data, requires_grad=False),np.array(Train_label, requires_grad=True)

In [20]:
label_0 = [[np.sqrt(2), 0, 0, 0],
        [0, 1, 1, 0],
        [0, 1, -1, 0],
        [0, 0, 0, np.sqrt(2)]]
label_1 = [[np.sqrt(2), 0, 0, 0],
        [0, -1, 1, 0],
        [0, 1, 1, 0],
        [0, 0, 0, np.sqrt(2)]]
state_labels = np.array([label_0, label_1], requires_grad=False)
def density_matrix(state):
#     return state * np.conj(state).T
    return state/np.sqrt(2)

@qml.qnode(dev, interface="autograd")
def qcircuit(params, x, y):
    

    
    embedding_layer(x)
#     XY_layer(10,params)
    XY_layer(10,params[:10])
    embedding_layer(x)
    XY_layer(10,params[10:20])
    embedding_layer(x)
    XY_layer(10,params[20:30])

    Y_pooling(10)
    
    XY_layer(5,params[30:35])
    Y_pooling(5)
    
    Y_pooling(4)
    
    qml.CY(wires=[4, 0])

    
    return qml.expval(qml.Hermitian(y, wires=[0,1]))

def cost(params, x, y, state_labels=None):
    # Compute prediction for each input in data batch
    loss = 0.0
    dm_labels = [density_matrix(s) for s in state_labels]
    for i in range(len(x)):
        ypred = qcircuit(params, x[i], dm_labels[0])
        loss = loss  -y[i] * np.log(ypred, requires_grad=True) - (1 - y[i]) * np.log(1 - ypred, requires_grad=True)
    return loss / len(x)

def test(params, x, y, state_labels=None):
    fidelity_values = []
    dm_labels = [density_matrix(s) for s in state_labels]
    predicted = []

    for i in range(len(x)):
#         fidel_function = lambda y: qcircuit(params, x[i], y)
        fidelities = qcircuit(params, x[i], dm_labels[0])
        best_fidel = np.rint(fidelities)

        predicted.append(best_fidel)
        fidelity_values.append(fidelities)

    return np.array(predicted), np.array(fidelity_values)


def accuracy_score(y_true, y_pred):
    score = y_true == y_pred
    return score.sum() / len(y_true)


def iterate_minibatches(inputs, targets, batch_size):
    for start_idx in range(0, inputs.shape[0] - batch_size + 1, batch_size):
        idxs = slice(start_idx, start_idx + batch_size)
        yield inputs[idxs], targets[idxs]

In [21]:
learning_rate = 5e-3
epochs = 40
batch_size = 32



# # initialize random weights
params = np.random.uniform(size=35, requires_grad=True)
# best_params = np.load('best_params_task_VII.npy')
# params = best_params

opt = AdamOptimizer(learning_rate, beta1=0.9, beta2=0.999)

best_val_accuracy = 0

for it in range(epochs):
#     print(params[:10])
    for Xbatch, ybatch in iterate_minibatches(Train_data, Train_label, batch_size=batch_size):
        params, _, _, _ = opt.step(cost, params, Xbatch, ybatch, state_labels)
    
    predicted_train, fidel_train = test(params, Train_data, Train_label, state_labels)
    accuracy_train = accuracy_score(Train_label, predicted_train)
    loss = cost(params, Train_data, Train_label, state_labels)
    
    predicted_test, fidel_test = test(params, Test_data, Test_label, state_labels)
    accuracy_test = accuracy_score(Test_label, predicted_test)
    
    if accuracy_test > best_val_accuracy:
        best_params = params.copy()
        best_val_accuracy = accuracy_test
    res = [it + 1, loss, accuracy_train, accuracy_test]
    print(
        "Epoch: {:2d} | Loss: {:3f} | Train accuracy: {:3f} | Test accuracy: {:3f}".format(
            *res
        )
    )

Epoch:  1 | Loss: 0.716095 | Train accuracy: 0.518571 | Test accuracy: 0.530000
Epoch:  2 | Loss: 0.710201 | Train accuracy: 0.527143 | Test accuracy: 0.570000
Epoch:  3 | Loss: 0.706640 | Train accuracy: 0.522857 | Test accuracy: 0.600000
Epoch:  4 | Loss: 0.704093 | Train accuracy: 0.524286 | Test accuracy: 0.630000
Epoch:  5 | Loss: 0.702161 | Train accuracy: 0.527143 | Test accuracy: 0.630000
Epoch:  6 | Loss: 0.700630 | Train accuracy: 0.527143 | Test accuracy: 0.640000
Epoch:  7 | Loss: 0.699375 | Train accuracy: 0.527143 | Test accuracy: 0.650000
Epoch:  8 | Loss: 0.698317 | Train accuracy: 0.534286 | Test accuracy: 0.640000
Epoch:  9 | Loss: 0.697402 | Train accuracy: 0.540000 | Test accuracy: 0.610000
Epoch: 10 | Loss: 0.696592 | Train accuracy: 0.538571 | Test accuracy: 0.600000
Epoch: 11 | Loss: 0.695859 | Train accuracy: 0.537143 | Test accuracy: 0.600000
Epoch: 12 | Loss: 0.695186 | Train accuracy: 0.542857 | Test accuracy: 0.590000
Epoch: 13 | Loss: 0.694558 | Train accur